In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"oliverli0414","key":"19df8e5218b8495a8c1ea362134c4a27"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c home-credit-default-risk
!unzip home-credit-default-risk.zip

 99% 681M/688M [00:07<00:00, 135MB/s]
100% 688M/688M [00:07<00:00, 93.4MB/s]
Archive:  home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
application_train = pd.read_csv('application_train.csv')[['SK_ID_CURR','TARGET']]
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')

In [ ]:
print(bureau.isna().sum())
print(bureau_balance.isna().sum())
#print(bureau['CREDIT_ACTIVE'].value_counts())
#print(previous_application['SK_ID_CURR'].nunique())
print(bureau.shape[0])
#print(previous_application['CODE_REJECT_REASON'].value_counts())
#print(previous_application['NAME_YIELD_GROUP'].value_counts())

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
dtype: int64
SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64
1716428


In [71]:
bureau_balance_sorted = pd.DataFrame()
bureau_balance_sorted['BALANCE_COUNT'] = -bureau_balance.groupby('SK_ID_BUREAU').min('MONTHS_BALANCE')+1
bureau_balance_sorted['OCCURENCES']=bureau_balance.groupby('SK_ID_BUREAU').count()['MONTHS_BALANCE']
bureau_balance_sorted['COUNT_C']=bureau_balance[bureau_balance['STATUS']=='C'].groupby('SK_ID_BUREAU')['STATUS'].count()
bureau_balance_sorted['COUNT_X']=bureau_balance[bureau_balance['STATUS']=='X'].groupby('SK_ID_BUREAU')['STATUS'].count()
bureau_balance_sorted['COUNT_0']=bureau_balance[bureau_balance['STATUS']=='0'].groupby('SK_ID_BUREAU')['STATUS'].count()
bureau_balance_sorted['COUNT_1']=bureau_balance[bureau_balance['STATUS']=='1'].groupby('SK_ID_BUREAU')['STATUS'].count()
bureau_balance_sorted['COUNT_2']=bureau_balance[bureau_balance['STATUS']=='2'].groupby('SK_ID_BUREAU')['STATUS'].count()
bureau_balance_sorted['COUNT_3']=bureau_balance[bureau_balance['STATUS']=='3'].groupby('SK_ID_BUREAU')['STATUS'].count()
bureau_balance_sorted['COUNT_4']=bureau_balance[bureau_balance['STATUS']=='4'].groupby('SK_ID_BUREAU')['STATUS'].count()
bureau_balance_sorted['COUNT_5']=bureau_balance[bureau_balance['STATUS']=='5'].groupby('SK_ID_BUREAU')['STATUS'].count()
bureau_balance_sorted=bureau_balance_sorted.fillna(0)
bureau_balance_sorted['COUNT_ge1']=bureau_balance_sorted['COUNT_1']+bureau_balance_sorted['COUNT_2']+bureau_balance_sorted['COUNT_3']+bureau_balance_sorted['COUNT_4']+bureau_balance_sorted['COUNT_5']

In [72]:
print(bureau_balance_sorted)

              BALANCE_COUNT  OCCURENCES  COUNT_C  COUNT_X  COUNT_0  COUNT_1  \
SK_ID_BUREAU                                                                  
5001709                  97          97     86.0     11.0      0.0      0.0   
5001710                  83          83     48.0     30.0      5.0      0.0   
5001711                   4           4      0.0      1.0      3.0      0.0   
5001712                  19          19      9.0      0.0     10.0      0.0   
5001713                  22          22      0.0     22.0      0.0      0.0   
...                     ...         ...      ...      ...      ...      ...   
6842884                  48          48     20.0     19.0      9.0      0.0   
6842885                  24          24      0.0      0.0     12.0      0.0   
6842886                  33          33     25.0      0.0      8.0      0.0   
6842887                  37          37     31.0      0.0      6.0      0.0   
6842888                  62          62     52.0    

In [ ]:
bureau_merge = bureau.merge(bureau_balance_sorted, on='SK_ID_BUREAU', how='left')

In [ ]:
bureau_sorted = pd.DataFrame()
bureau_sorted['COUNT_ID'] = bureau_merge.groupby('SK_ID_CURR').count()['SK_ID_BUREAU']
bureau_sorted['COUNT_ACTIVE'] = bureau_merge[bureau_merge['CREDIT_ACTIVE']=='Active'].groupby('SK_ID_CURR')['CREDIT_ACTIVE'].count()
bureau_sorted['COUNT_CLOSED'] = bureau_merge[bureau_merge['CREDIT_ACTIVE']=='Closed'].groupby('SK_ID_CURR')['CREDIT_ACTIVE'].count()
bureau_sorted['COUNT_BADDEBT'] = bureau_merge[bureau_merge['CREDIT_ACTIVE']=='Bad debt'].groupby('SK_ID_CURR')['CREDIT_ACTIVE'].count()
bureau_sorted['COUNT_SOLD'] = bureau_merge[bureau_merge['CREDIT_ACTIVE']=='Sold'].groupby('SK_ID_CURR')['CREDIT_ACTIVE'].count()
bureau_sorted[['COUNT_ACTIVE','COUNT_CLOSED','COUNT_BADDEBT','COUNT_SOLD']]=bureau_sorted[['COUNT_ACTIVE','COUNT_CLOSED','COUNT_BADDEBT','COUNT_SOLD']].fillna(0)
bureau_sorted['DISTINCT_CURRENCIES'] = bureau_merge.groupby('SK_ID_CURR')['CREDIT_CURRENCY'].nunique()
bureau_sorted['SUM_DAYS_CREDIT'] = bureau_merge.groupby('SK_ID_CURR')['DAYS_CREDIT'].sum()
bureau_sorted['AVG_DAYS_CREDIT'] = bureau_sorted['SUM_DAYS_CREDIT']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_CREDIT_DAY_OVERDUE'] = bureau_merge.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].sum()
bureau_sorted['AVG_CREDIT_DAY_OVERDUE'] = bureau_sorted['SUM_CREDIT_DAY_OVERDUE']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_DAYS_CREDIT_ENDDATE'] = bureau_merge.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].sum()
bureau_sorted['AVG_DAYS_CREDIT_ENDDATE'] = bureau_sorted['SUM_DAYS_CREDIT_ENDDATE']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_DAYS_ENDDATE_FACT'] = bureau_merge.groupby('SK_ID_CURR')['DAYS_ENDDATE_FACT'].sum()
bureau_sorted['AVG_DAYS_ENDDATE_FACT'] = bureau_sorted['SUM_DAYS_ENDDATE_FACT']/ bureau_sorted['COUNT_CLOSED']
bureau_sorted['SUM_AMT_CREDIT_MAX_OVERDUE'] = bureau_merge.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].sum()
bureau_sorted['AVG_AMT_CREDIT_MAX_OVERDUE'] = bureau_sorted['SUM_AMT_CREDIT_MAX_OVERDUE']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_CNT_CREDIT_PROLONG'] = bureau_merge.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].sum()
bureau_sorted['AVG_CNT_CREDIT_PROLONG'] = bureau_sorted['SUM_CNT_CREDIT_PROLONG']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_AMT_CREDIT_SUM'] = bureau_merge.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].sum()
bureau_sorted['AVG_AMT_CREDIT_SUM'] = bureau_sorted['SUM_AMT_CREDIT_SUM']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_AMT_CREDIT_SUM_DEBT'] = bureau_merge.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_DEBT'].sum()
bureau_sorted['AVG_AMT_CREDIT_SUM_DEBT'] = bureau_sorted['SUM_AMT_CREDIT_SUM_DEBT']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_AMT_CREDIT_SUM_LIMIT'] = bureau_sorted['SUM_AMT_CREDIT_SUM'] - bureau_sorted['SUM_AMT_CREDIT_SUM_DEBT']
#bureau_sorted['SUM_AMT_CREDIT_SUM_LIMIT'] = bureau_merge.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_LIMIT'].sum()
bureau_sorted['AVG_AMT_CREDIT_SUM_LIMIT'] = bureau_sorted['SUM_AMT_CREDIT_SUM_LIMIT']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_AMT_CREDIT_SUM_OVERDUE'] = bureau_merge.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].sum()
bureau_sorted['AVG_AMT_CREDIT_SUM_OVERDUE'] = bureau_sorted['SUM_AMT_CREDIT_SUM_OVERDUE']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_DAYS_CREDIT_UPDATE'] = bureau_merge.groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].sum()
bureau_sorted['AVG_DAYS_CREDIT_UPDATE'] = bureau_sorted['SUM_DAYS_CREDIT_UPDATE']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_AMT_ANNUITY'] = bureau_merge.groupby('SK_ID_CURR')['AMT_ANNUITY'].sum()
bureau_sorted['AVG_AMT_ANNUITY'] = bureau_sorted['SUM_AMT_ANNUITY']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_C'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_C'].sum()
bureau_sorted['AVG_COUNT_C'] = bureau_sorted['SUM_COUNT_C']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_X'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_X'].sum()
bureau_sorted['AVG_COUNT_X'] = bureau_sorted['SUM_COUNT_X']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_0'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_0'].sum()
bureau_sorted['AVG_COUNT_0'] = bureau_sorted['SUM_COUNT_0']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_1'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_1'].sum()
bureau_sorted['AVG_COUNT_1'] = bureau_sorted['SUM_COUNT_1']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_2'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_2'].sum()
bureau_sorted['AVG_COUNT_2'] = bureau_sorted['SUM_COUNT_2']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_3'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_3'].sum()
bureau_sorted['AVG_COUNT_3'] = bureau_sorted['SUM_COUNT_3']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_4'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_4'].sum()
bureau_sorted['AVG_COUNT_4'] = bureau_sorted['SUM_COUNT_4']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_5'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_5'].sum()
bureau_sorted['AVG_COUNT_5'] = bureau_sorted['SUM_COUNT_5']/ bureau_sorted['COUNT_ID']
bureau_sorted['SUM_COUNT_ge1'] = bureau_merge.groupby('SK_ID_CURR')['COUNT_ge1'].sum()
bureau_sorted['AVG_COUNT_ge1'] = bureau_sorted['SUM_COUNT_ge1']/ bureau_sorted['COUNT_ID']


In [ ]:
print(bureau_sorted)

            COUNT_ID  COUNT_ACTIVE  COUNT_CLOSED  COUNT_BADDEBT  COUNT_SOLD  \
SK_ID_CURR                                                                    
100001             7           3.0           4.0            0.0         0.0   
100002             8           2.0           6.0            0.0         0.0   
100003             4           1.0           3.0            0.0         0.0   
100004             2           0.0           2.0            0.0         0.0   
100005             3           2.0           1.0            0.0         0.0   
...              ...           ...           ...            ...         ...   
456249            13           2.0          11.0            0.0         0.0   
456250             3           2.0           1.0            0.0         0.0   
456253             4           2.0           2.0            0.0         0.0   
456254             1           0.0           1.0            0.0         0.0   
456255            11           5.0           6.0    

In [ ]:
bureau_sorted__application_train = bureau_sorted.merge(application_train, on='SK_ID_CURR', how='inner')
print(bureau_sorted__application_train)

        SK_ID_CURR  COUNT_ID  COUNT_ACTIVE  COUNT_CLOSED  COUNT_BADDEBT  \
0           100002         8           2.0           6.0            0.0   
1           100003         4           1.0           3.0            0.0   
2           100004         2           0.0           2.0            0.0   
3           100007         1           0.0           1.0            0.0   
4           100008         3           1.0           2.0            0.0   
...            ...       ...           ...           ...            ...   
263486      456247        11           3.0           8.0            0.0   
263487      456249        13           2.0          11.0            0.0   
263488      456253         4           2.0           2.0            0.0   
263489      456254         1           0.0           1.0            0.0   
263490      456255        11           5.0           6.0            0.0   

        COUNT_SOLD  DISTINCT_CURRENCIES  SUM_DAYS_CREDIT  AVG_DAYS_CREDIT  \
0              0.0    

In [ ]:
corr = bureau_sorted__application_train.corr(method='spearman')
print(corr)

                            SK_ID_CURR  COUNT_ID  COUNT_ACTIVE  COUNT_CLOSED  \
SK_ID_CURR                    1.000000  0.003734      0.003294      0.002970   
COUNT_ID                      0.003734  1.000000      0.686290      0.884025   
COUNT_ACTIVE                  0.003294  0.686290      1.000000      0.307179   
COUNT_CLOSED                  0.002970  0.884025      0.307179      1.000000   
COUNT_BADDEBT                 0.001295  0.006622      0.004692      0.002114   
COUNT_SOLD                   -0.005728  0.099267      0.046085      0.062460   
DISTINCT_CURRENCIES           0.003289  0.043845      0.027184      0.042895   
SUM_DAYS_CREDIT              -0.001470 -0.874924     -0.445949     -0.879361   
AVG_DAYS_CREDIT               0.002946 -0.228998      0.149767     -0.402199   
SUM_CREDIT_DAY_OVERDUE        0.001592  0.055068      0.082414      0.019880   
AVG_CREDIT_DAY_OVERDUE        0.001592  0.054771      0.082226      0.019584   
SUM_DAYS_CREDIT_ENDDATE       0.001178 -

In [ ]:
corr.to_excel(r'bureau_inner.spearman.xlsx',index=True,header=True)

Choose COUNT_ACTIVE, COUNT_CLOSED, AVG_DAYS_CREDIT, AVG_CREDIT_DAY_OVERDUE, AVG_DAYS_CREDIT_ENDDATE, AMT_CREDIT_MAX_OVERDUE, AMT_CREDIT_SUM - AMT_CREDIT_SUM_DEBT AVG_AMT_CREDIT_SUM_DEBT, AVG_COUNT_ge1 due to correlation and intuitive sense to Target